In [1]:
import numpy as np
import pandas as pd
import pickle
import sys
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM, GRU
from keras.layers.wrappers import TimeDistributed
from keras.layers.core import RepeatVector
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.layers import Dense, Dropout, Activation
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from keras.utils import np_utils
from keras.optimizers import SGD, Adam, Adadelta
from keras.models import model_from_json

Using TensorFlow backend.


In [2]:
mypath = '/Users/Lucy/Google Drive/MSDS/2016Fall/DSGA1006/Data/rnn-encoder-decoder'
f = open(mypath + '/crunchbase.train.10000.pickle','rb')
train_x, train_y = pickle.load(f)
f.close()
f = open(mypath + '/crunchbase.test.10000.pickle','rb')
test_x, test_y = pickle.load(f)
f.close()

In [19]:
def pad(x, length = 100):
	# truncate and pad input sequences
	max_desc_length = length
	train_x = sequence.pad_sequences(x, maxlen=max_desc_length)

	return train_x

def transform_data(x, top_words = 10000):
	dim = x.shape
	transformed = np.zeros((dim[0],dim[1],top_words+1))
	for i,seq in enumerate(x):
		for j,word in enumerate(seq):
			transformed[i][j][word] = 1

	return transformed

In [5]:
train_x = pad(train_x, 100)
test_x = pad(test_x, 100)
train_y = pad(train_y, 15)
test_y = pad(test_y, 15)

In [13]:
train_x[7]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    2,    1,   60,   10,
          7,  132,    8,  140, 2042, 2248,  893,   66,   64,    4,   40,
        310,    4,  219,   33,    3,    2,   57,    9,  828,    4,   42,
          9,    1, 7175,    1,   10, 7481,   12,  437,   23,  103,   26,
         68,  108,    1,    1, 4518,    1,    1,  168, 6793,    4,    1,
        719], dtype=int32)

In [21]:
dim = train_x.shape

In [24]:
transformed = np.zeros((dim[0],dim[1],10001))

In [25]:
for i,seq in enumerate(train_x):
    for j,word in enumerate(seq):
        transformed[i][j][word] = 1

In [32]:
transformed[0][99]

array([ 0.,  0.,  0., ...,  0.,  0.,  0.])

In [16]:
vocabulary_size = 5000
unknown_token = "UNKNOWNTOKEN"
sentence_start_token = "SENTENCESTART"
sentence_end_token = "SENTENCEEND"

In [17]:
# Split full descriptions into sentences
sentences = itertools.chain(*[[nltk.sent_tokenize(x.decode('utf-8').lower())] for x in train_description])
mod_sentences = []
for desc in sentences:
    full_desc = ""
    for x in desc:
        s = "%s %s %s " % (sentence_start_token, x, sentence_end_token)
        full_desc = full_desc + s
    mod_sentences.append(full_desc)

In [18]:
mod_sentences[:2]

[u'SENTENCESTART a smart glove that helps workers in logistics and manufacturing work more efficiently.a smart glove that helps workers in logistics and manufacturing work more efficiently. SENTENCEEND ',
 u'SENTENCESTART passlogix provides fast and secure access to enterprise resources with their flagship product; the v-go access accelerator suite.passlogix provides fast and secure access to enterprise resources with their flagship product; the v-go access accelerator suite. SENTENCEEND ']

In [19]:
def strip_punctuation(text):
    """
    >>> strip_punctuation(u'something')
    u'something'

    >>> strip_punctuation(u'something.,:else really')
    u'somethingelse really'
    """
    punctutation_cats = set(['Pc', 'Pd', 'Ps', 'Pe', 'Pi', 'Pf', 'Po'])
    return ''.join(x for x in text
                   if unicodedata.category(x) not in punctutation_cats)

In [20]:
mod_sentences = map(strip_punctuation, mod_sentences)

In [21]:
for i in mod_sentences[:5]:
    print i + '\n'

SENTENCESTART a smart glove that helps workers in logistics and manufacturing work more efficientlya smart glove that helps workers in logistics and manufacturing work more efficiently SENTENCEEND 

SENTENCESTART passlogix provides fast and secure access to enterprise resources with their flagship product the vgo access accelerator suitepasslogix provides fast and secure access to enterprise resources with their flagship product the vgo access accelerator suite SENTENCEEND 

SENTENCESTART were on a mission to provide reliable solar energy for schools hospitals and water projects around the worldwere on a mission to provide reliable solar energy for schools hospitals and water projects around the world SENTENCEEND 

SENTENCESTART haus bioceuticals is a specialty pharmaceutical company developing clinically active botanical extractshaus bioceuticals is a specialty pharmaceutical company developing clinically active botanical extracts SENTENCEEND 

SENTENCESTART kivera foresaw the emergen

In [22]:
tokenized_sentences = [nltk.word_tokenize(sent) for sent in mod_sentences]

In [23]:
for i in tokenized_sentences[3]:
    print i

SENTENCESTART
haus
bioceuticals
is
a
specialty
pharmaceutical
company
developing
clinically
active
botanical
extractshaus
bioceuticals
is
a
specialty
pharmaceutical
company
developing
clinically
active
botanical
extracts
SENTENCEEND


In [25]:
train_y[3]

"Lydia's iPhone and Android apps enable you to pay anyone very easily and for free."

In [26]:
# Count the word frequencies
word_freq = nltk.FreqDist(itertools.chain(*tokenized_sentences))
print "Found %d unique words tokens." % len(word_freq.items())

Found 109713 unique words tokens.


In [27]:
#word_freq = sorted(word_freq.items(), key=lambda i: i[1], reverse=True)

In [28]:
# Get the most common words and build index_to_word and word_to_index vectors
vocab = word_freq.most_common(vocabulary_size-1)
index_to_word = [x[0] for x in vocab]
index_to_word.append(unknown_token)
word_to_index = dict([(w,i) for i,w in enumerate(index_to_word)])

print "Using vocabulary size %d." % vocabulary_size
print "The least frequent word in our vocabulary is '%s' and appeared %d times." % (vocab[-1][0], vocab[-1][1])

# Replace all words not in our vocabulary with the unknown token
for i, sent in enumerate(tokenized_sentences):
    tokenized_sentences[i] = [w if w in word_to_index else unknown_token for w in sent]
    


Using vocabulary size 5000.
The least frequent word in our vocabulary is 'hold' and appeared 16 times.


In [29]:
{word_to_index.keys()[word_to_index.values().index(k)]: k for k in sorted(word_to_index.values())[:2]}

{u'SENTENCEEND': 1, u'and': 0}

In [32]:
np.array(tokenized_sentences).shape

(50691,)